# File konfigurasi.py

In [ ]:
import os

BASE_DIR = os.path.dirname(os.path.abspath(__file__))
NAMA_DB = 'fasilitas_kesehatan.db'
DB_PATH = os.path.join(BASE_DIR, 'data', NAMA_DB)

TIPE_FASKES = ["Rumah Sakit", "Puskesmas", "Klinik", "Apotek"]

# File model.py

In [ ]:
from abc import ABC, abstractmethod

class FasilitasKesehatan(ABC):
    """Kelas abstrak untuk merepresentasikan fasilitas kesehatan."""
    def __init__(self, nama: str, latitude: float, longitude: float, alamat: str):
        self.nama = str(nama) if nama else "Tanpa Nama"
        try:
            self.latitude = float(latitude)
        except (ValueError, TypeError):
            self.latitude = 0.0
            print(f"Peringatan: Latitude '{latitude}' tidak valid.")

        try:
            self.longitude = float(longitude)
        except (ValueError, TypeError):
            self.longitude = 0.0
            print(f"Peringatan: Longitude '{longitude}' tidak valid.")

        self.alamat = str(alamat) if alamat else "-"

    def get_koordinat(self) -> tuple:
        return (self.latitude, self.longitude)

    @abstractmethod
    def get_info_popup(self) -> str:
        pass

    def __repr__(self) -> str:
        return f"Faskes('{self.nama}', Tipe:{self.__class__.__name__})"

class RumahSakit(FasilitasKesehatan):
    def get_info_popup(self) -> str:
        return f"<b>{self.nama}</b><br><i>Rumah Sakit</i><br>{self.alamat}"

class Puskesmas(FasilitasKesehatan):
    def get_info_popup(self) -> str:
        return f"<b>{self.nama}</b><br><i>Puskesmas</i><br>{self.alamat}"

class Klinik(FasilitasKesehatan):
    def get_info_popup(self) -> str:
        return f"<b>{self.nama}</b><br><i>Klinik</i><br>{self.alamat}"

class Apotek(FasilitasKesehatan):
    def get_info_popup(self) -> str:
        return f"<b>{self.nama}</b><br><i>Apotek</i><br>{self.alamat}"


# File database.py

In [ ]:
import sqlite3
import pandas as pd
from konfigurasi import DB_PATH

def get_db_connection() -> sqlite3.Connection | None:
    try:
        conn = sqlite3.connect(DB_PATH, timeout=10, detect_types=sqlite3.PARSE_DECLTYPES)
        conn.row_factory = sqlite3.Row
        return conn
    except sqlite3.Error as e:
        print(f"ERROR [database.py] Koneksi DB gagal: {e}")
        return None

def execute_query(query: str, params: tuple = None):
    conn = get_db_connection()
    if not conn:
        return None
    try:
        cursor = conn.cursor()
        cursor.execute(query, params or ())
        conn.commit()
        return cursor.lastrowid
    except sqlite3.Error as e:
        print(f"ERROR [database.py] Query gagal: {e} | Query: {query[:60]}")
        conn.rollback()
        return None
    finally:
        conn.close()

def fetch_query(query: str, params: tuple = None, fetch_all: bool = True):
    conn = get_db_connection()
    if not conn:
        return None
    try:
        cursor = conn.cursor()
        cursor.execute(query, params or ())
        return cursor.fetchall() if fetch_all else cursor.fetchone()
    except sqlite3.Error as e:
        print(f"ERROR [database.py] Fetch gagal: {e} | Query: {query[:60]}")
        return None
    finally:
        conn.close()

def get_dataframe(query: str, params: tuple = None) -> pd.DataFrame:
    conn = get_db_connection()
    if not conn:
        return pd.DataFrame()
    try:
        return pd.read_sql_query(query, conn, params=params)
    except Exception as e:
        print(f"ERROR [database.py] Gagal baca ke DataFrame: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

def setup_database_initial():
    conn = get_db_connection()
    if not conn:
        return False
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS faskes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nama TEXT NOT NULL,
                tipe TEXT NOT NULL,
                alamat TEXT,
                latitude REAL,
                longitude REAL
            );
        """)
        conn.commit()
        return True
    except sqlite3.Error as e:
        print(f"ERROR saat setup tabel: {e}")
        return False
    finally:
        conn.close()

# File setup_db.py

In [ ]:
import sqlite3
import os
from konfigurasi import DB_PATH

def setup_database():
    print(f"Memeriksa/membuat database fasilitas kesehatan di: {DB_PATH}")
    conn = None
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        sql_create_table = """
        CREATE TABLE IF NOT EXISTS faskes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nama TEXT NOT NULL,
            tipe TEXT NOT NULL,
            alamat TEXT,
            latitude REAL,
            longitude REAL
        );"""
        print("Membuat tabel 'faskes' (jika belum ada)...")
        cursor.execute(sql_create_table)
        conn.commit()
        print("-> Tabel 'faskes' siap.")
        return True
    except sqlite3.Error as e:
        print(f"-> Error SQLite saat setup: {e}")
        return False
    finally:
        if conn:
            conn.close()
            print("-> Koneksi DB setup ditutup.")

if __name__ == "__main__":
    print("--- Memulai Setup Database Fasilitas Kesehatan ---")
    os.makedirs("data", exist_ok=True)
    if setup_database():
        print(f"\nSetup database '{os.path.basename(DB_PATH)}' selesai.")
    else:
        print("\nSetup database GAGAL.")
    print("--- Setup Database Selesai ---")


# File manajer_faskes.py

In [ ]:
from model import RumahSakit, Puskesmas, Klinik, Apotek
import database

class ManajerFaskes:
    _db_setup_done = False

    def __init__(self):
        if not ManajerFaskes._db_setup_done:
            print("[ManajerFaskes] Menyiapkan database fasilitas kesehatan...")
            if database.setup_database_initial():
                ManajerFaskes._db_setup_done = True
                print("[ManajerFaskes] Database siap digunakan.")
            else:
                print("[ManajerFaskes] GAGAL menyiapkan database.")

    def tambah_faskes(self, nama, tipe, alamat, lat, lon) -> bool:
        if not nama or not tipe:
            return False
        query = "INSERT INTO faskes (nama, tipe, alamat, latitude, longitude) VALUES (?, ?, ?, ?, ?)"
        return database.execute_query(query, (nama, tipe, alamat, lat, lon)) is not None

    def hapus_faskes(self, nama: str) -> bool:
        return database.execute_query("DELETE FROM faskes WHERE nama = ?", (nama,)) is not None

    def ambil_dataframe_faskes(self):
        return database.get_dataframe("SELECT * FROM faskes ORDER BY nama")

    def buat_objek_faskes(self, row) -> object:
        tipe = row['tipe']
        nama, lat, lon, alamat = row['nama'], row['latitude'], row['longitude'], row['alamat']
        if tipe == "Rumah Sakit":
            return RumahSakit(nama, lat, lon, alamat)
        elif tipe == "Puskesmas":
            return Puskesmas(nama, lat, lon, alamat)
        elif tipe == "Klinik":
            return Klinik(nama, lat, lon, alamat)
        elif tipe == "Apotek":
            return Apotek(nama, lat, lon, alamat)
        return None

    def ambil_semua_objek_faskes(self) -> list:
        rows = database.fetch_query("SELECT * FROM faskes ORDER BY nama")
        return [self.buat_objek_faskes(row) for row in rows if row]

# File main_app.py

In [ ]:
# main_app.py - Aplikasi Web Faskes Semarang

import streamlit as st
import pandas as pd
import folium
import os

# --- Import modul internal ---
try:
    from manajer_faskes import ManajerFaskes
    from konfigurasi import TIPE_FASKES
except ImportError as e:
    st.error(f"Gagal mengimpor modul: {e}")
    st.stop()

from streamlit_folium import st_folium

# --- Inisialisasi manajer faskes ---
manajer = ManajerFaskes()

# --- Konfigurasi halaman ---
st.set_page_config(
    page_title="Fasilitas Kesehatan Semarang",
    layout="wide",
    initial_sidebar_state="expanded"
)

# =========================
#       HALAMAN: INPUT
# =========================
def halaman_input():
    st.header("➕ Tambah Fasilitas Kesehatan")

    with st.form("form_faskes", clear_on_submit=True):
        nama = st.text_input("Nama Faskes")
        tipe = st.selectbox("Tipe Faskes", TIPE_FASKES)
        alamat = st.text_input("Alamat")

        col1, col2 = st.columns(2)
        lat = col1.number_input("Latitude", format="%.6f")
        lon = col2.number_input("Longitude", format="%.6f")

        submit = st.form_submit_button("💾 Simpan")

        if submit:
            if not nama or not tipe:
                st.warning("Nama dan tipe wajib diisi.", icon="⚠️")
            else:
                manajer.tambah_faskes(nama, tipe, alamat, lat, lon)
                st.success(f"'{nama}' berhasil ditambahkan.")
                st.rerun()

# =========================
#       HALAMAN: HAPUS
# =========================
def halaman_hapus():
    st.header("🗑 Hapus Fasilitas Kesehatan")
    df = manajer.ambil_dataframe_faskes()

    if df.empty:
        st.info("Belum ada data untuk dihapus.")
        return

    # Pilih data yang ingin dihapus
    nama_hapus = st.selectbox("Pilih fasilitas yang akan dihapus", df["nama"].tolist())

    # Tombol hapus pertama (untuk memunculkan konfirmasi)
    if 'hapus_ditekan' not in st.session_state:
        st.session_state.hapus_ditekan = False

    if not st.session_state.hapus_ditekan:
        if st.button("🗑 Hapus"):
            st.session_state.hapus_ditekan = True
            st.rerun()
    else:
        st.warning(f"Anda yakin ingin menghapus **{nama_hapus}**?", icon="⚠️")
        konfirmasi = st.checkbox("Saya yakin ingin menghapus data ini.")
        if st.button("✅ Ya, Hapus Sekarang"):
            if konfirmasi:
                sukses = manajer.hapus_faskes(nama_hapus)
                if sukses:
                    st.success(f"'{nama_hapus}' berhasil dihapus.")
                    st.session_state.hapus_ditekan = False
                    st.rerun()
                else:
                    st.error("❌ Gagal menghapus data.")
            else:
                st.warning("✅ Silakan centang konfirmasi terlebih dahulu.")

# =========================
#     HALAMAN: RINGKASAN
# =========================
def halaman_ringkasan():
    st.header("🌏 Ringkasan & Peta Fasilitas Kesehatan")
    df = manajer.ambil_dataframe_faskes()

    if df.empty:
        st.info("Belum ada data untuk ditampilkan.")
        return

    # --- Filter Tipe ---
    daftar_tipe = ["Semua"] + sorted(df["tipe"].unique().tolist())
    tipe_filter = st.selectbox("Filter berdasarkan tipe:", daftar_tipe)

    if tipe_filter != "Semua":
        df = df[df["tipe"] == tipe_filter]

    st.dataframe(df, use_container_width=True)

    # --- Buat Peta ---
    faskes_list = [manajer.buat_objek_faskes(row) for _, row in df.iterrows()]
    peta = folium.Map(location=[-7.029374387601962, 110.37953560884178], zoom_start=12) # Koordinat rumah saya hehe

    warna_marker = {
        "RumahSakit": "red",
        "Puskesmas": "blue",
        "Klinik": "pink",
        "Apotek": "green"
    }

    for f in faskes_list:
        if f:
            warna = warna_marker.get(type(f).__name__, "gray")
            folium.Marker(
                location=f.get_koordinat(),
                popup=f.get_info_popup(),
                tooltip=f.nama,
                icon=folium.Icon(color=warna)
            ).add_to(peta)

    st_folium(peta, width=700, height=500)

    os.makedirs("output", exist_ok=True)
    peta.save("output/peta_faskes.html")

# =========================
#           MAIN
# =========================
def main():
    st.sidebar.title("🏥 Faskes Semarang")
    menu = st.sidebar.radio("Pilih Menu:", ["Tambah", "Hapus", "Ringkasan"])

    if menu == "Tambah":
        halaman_input()
    elif menu == "Hapus":
        halaman_hapus()
    elif menu == "Ringkasan":
        halaman_ringkasan()

    st.sidebar.markdown("---")
    st.sidebar.caption("Sistem Informasi Fasilitas Kesehatan di Semarang")
    st.sidebar.caption("Septia Isnaeni Salsabila😼")

if __name__ == "__main__":
    main()